In [10]:
from siphon.catalog import TDSCatalog
import xarray as xr
import rioxarray as rxr

from google.cloud import storage
from datetime import datetime, date, timedelta
import os

In [11]:
os.environ['WEST'] = '90.0'
os.environ['EAST'] = '150.0'
os.environ['SOUTH'] = '-15.0'
os.environ['NORTH'] = '15.0'
os.environ['VARIABLE'] = 'daymean_cnc_PM2_5'

In [12]:
# Observation time
obs = date.today()
obs_time = datetime(obs.year, obs.month, obs.day, 12, 30)

URL = 'http://silam.fmi.fi/thredds/catalog/dailysilam_glob_v5_7_1/catalog.xml'
catalog = TDSCatalog(URL)

In [13]:
# NetCDF Subset Service
ncss = catalog.datasets[0].subset()

# Query boundary box and variable
variable = os.environ.get('VARIABLE')
west = float(os.environ.get('WEST'))
east = float(os.environ.get('EAST'))
south = float(os.environ.get('SOUTH'))
north = float(os.environ.get('NORTH'))

# Create subset query
query = ncss.query()
query.variables(variable).time(obs)
query.lonlat_box(west=west, east=east, south=south, north=north)
data = xr.backends.NetCDF4DataStore(ncss.get_data(query))

ds = xr.open_dataset(data).isel(time = 0) \
    .isel(hybrid = 0, drop = True)

In [14]:
import numpy as np

In [15]:
new_lon = np.linspace(ds.lon[0], ds.lon[-1], 60 * 20 + 1)
new_lat = np.linspace(ds.lat[0], ds.lat[-1], 30 * 20 + 1)

In [7]:
dsi = ds.interp(lat = new_lat, lon = new_lon)

In [8]:
date_str = obs.strftime('%Y%m%d')
fname = f'pm25_{date_str}_interp.tif'

concentration = dsi[variable]
concentration.rio.set_spatial_dims('lon', 'lat', inplace=True)
concentration.rio.set_crs(4326, inplace=True)
concentration.rio.to_raster(fname)

In [9]:
os.system(f'rio cogeo create {fname} {fname}')

client = storage.Client.from_service_account_json('neonet-sentinel-privatekey.json')
bucket = client.bucket('silam-neonet-rasters')
blob = bucket.blob(fname)
blob.upload_from_filename(fname)

os.remove(fname)

/home/josef/miniconda3/envs/geo/lib/python3.9/site-packages/rio_cogeo/cogeo.py:189: IncompatibleBlockRasterSize: Block Size are bigger than raster sizes. Setting blocksize to 256
  warnings.warn(
Reading input: /home/josef/Documents/Git/GeospatialPython/SILAM/pm25_20210718_interp.tif
Adding overviews...
Updating dataset tags...
Writing output to: /home/josef/Documents/Git/GeospatialPython/SILAM/pm25_20210718_interp.tif
